# Carregar a base de dados

In [1]:
import pandas as pd
import random 
import numpy as np 

O coronavírus (COVID-19) é uma doençaa infecciosa causada por um coronavírus
recém descoberta. A maioria das pessoas que adoece por causa da COVID-19 apresenta
sintomas leves a moderados e se recupera sem tratamento especial. 

Contudo, o coronavírus é uma doença com uma taxa de transmissão muito alta,
portanto gera um grande risco para a sociedade. Para combater a COVID-19 foi necessária
uma série de medidas pelo governo e pela sociedade: distanciamento social, aumento de
Unidades de Terapias Intensivas (UTIs), uso de máscara, fechamento de estabelecimentos,
toque de recolher nas cidades, lockdown.

O Governo do Distrito Federal, diariamente, em seu site, disponibiliza uma base de
dados sobre os infectados pela COVID-19


In [76]:
df = pd.read_csv('dados-abertos.csv',delimiter=';')

In [63]:
df.dropna(subset=['Faixa Etária'],inplace=True)
df.head().T

,0,1,2,3,4
Data,30/06/2021,30/06/2021,30/06/2021,30/06/2021,30/06/2021
Data Cadastro,2020-03-18,2020-03-18,2020-03-18,2020-03-18,2020-03-18
Sexo,Masculino,Masculino,Masculino,Masculino,Feminino
Faixa Etária,50 a 59 anos,50 a 59 anos,40 a 49 anos,>= 60 anos,20 a 29 anos
RA,Plano Piloto,Plano Piloto,Lago Sul,Águas Claras,Plano Piloto
UF,DISTRITO FEDERAL,DISTRITO FEDERAL,DISTRITO FEDERAL,DISTRITO FEDERAL,DISTRITO FEDERAL
Óbito,Não,Não,Não,Não,Não
dataPrimeirosintomas,23/04/2020,03/08/2020,09/03/2020,14/03/2020,14/03/2020
Idade,54,51,46,63,29
Pneumopatia,NaN,NaN,NaN,NaN,NaN


In [80]:
df.shape

(430461, 17)

# Tamanho da amostra

In [86]:
def tamanho_amostra(e,N):
  n = (N / (1 + (N*(e**2))))
  return (int(n))

e_ = 0.05
N_ = 430461

print(tamanho_amostra(e_, N_))

399


# Amostragem aleatória simples

In [8]:
def amostragem_aleatoria_simples(dataset, amostras):
  return dataset.sample(n = amostras, random_state=1)

In [87]:
df_amostra_aleatoria_simples = amostragem_aleatoria_simples(df, 399)
df_amostra_aleatoria_simples.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,UF,Óbito,dataPrimeirosintomas,Idade,Pneumopatia,Nefropatia,Doença Hematológica,Distúrbios Metabólicos,Imunopressão,Obesidade,Outros,Cardiovasculopatia
54354,30/06/2021,04/07/2020,Masculino,40 a 49 anos,Ceilândia,DISTRITO FEDERAL,Não,01/07/2020,46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
146260,30/06/2021,22/08/2020,Feminino,>= 60 anos,Planaltina,DISTRITO FEDERAL,Não,17/08/2020,60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
126892,30/06/2021,12/08/2020,Feminino,30 a 39 anos,Sobradinho,DISTRITO FEDERAL,Não,06/08/2020,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78365,30/06/2021,17/07/2020,Feminino,50 a 59 anos,Recanto das Emas,DISTRITO FEDERAL,Não,30/06/2020,52,Sim,NaN,NaN,Sim,NaN,NaN,NaN,Sim
78837,30/06/2021,18/07/2020,Feminino,40 a 49 anos,Guará,DISTRITO FEDERAL,Não,14/07/2020,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
df_amostra_aleatoria_simples.shape

(399, 17)

# Amostragem sistemática

In [21]:
def amostragem_sistematica(dataset, amostras):
  intervalo = len(dataset) // amostras
  random.seed(1)
  inicio = random.randint(0, intervalo)
  indices = np.arange(inicio, len(dataset), step = intervalo)
  amostra_sistematica = dataset.iloc[indices]
  return amostra_sistematica

In [88]:
df_amostra_sistematica = amostragem_sistematica(df, 399)
df_amostra_sistematica.shape

(400, 17)

# Amostragem por grupos

In [91]:
len(df)/1078.84962

399.00000150159946

In [35]:
def amostragem_agrupamento(df, numero_grupos):
  intervalo = len(df) / numero_grupos

  grupos = []
  id_grupo = 0
  contagem = 0
  for _ in df.iterrows():
    grupos.append(id_grupo)
    contagem += 1
    if contagem > intervalo:
      contagem = 0
      id_grupo += 1

  df['grupo'] = grupos
  random.seed(1)
  grupo_selecionado = random.randint(0, numero_grupos)
  return df[df['grupo'] == grupo_selecionado]

In [92]:
df_amostra_agrupamento = amostragem_agrupamento(df, 1078)
df_amostra_agrupamento.shape, df_amostra_agrupamento['grupo'].value_counts()

((400, 18), 275    400
 Name: grupo, dtype: int64)

In [93]:
df_amostra_agrupamento.head()

,Data,Data Cadastro,Sexo,Faixa Etária,RA,UF,Óbito,dataPrimeirosintomas,Idade,Pneumopatia,Nefropatia,Doença Hematológica,Distúrbios Metabólicos,Imunopressão,Obesidade,Outros,Cardiovasculopatia,grupo
110000,30/06/2021,03/08/2020,Feminino,<= 19 anos,Vicente Pires,DISTRITO FEDERAL,Não,02/07/2020,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275
110001,30/06/2021,03/08/2020,Feminino,30 a 39 anos,Gama,DISTRITO FEDERAL,Não,01/08/2020,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275
110002,30/06/2021,03/08/2020,Feminino,40 a 49 anos,Taguatinga,DISTRITO FEDERAL,Não,07/06/2020,42,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275
110003,30/06/2021,03/08/2020,Feminino,40 a 49 anos,Plano Piloto,DISTRITO FEDERAL,Não,31/07/2020,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275
110004,30/06/2021,03/08/2020,Feminino,30 a 39 anos,Plano Piloto,DISTRITO FEDERAL,Não,01/08/2020,37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,275


# Amostra estratificada

In [94]:
from sklearn.model_selection import StratifiedShuffleSplit

In [95]:
b = df['Faixa Etária'].value_counts()
print(b)

30 a 39 anos    106832
40 a 49 anos     95878
20 a 29 anos     72164
50 a 59 anos     63423
>= 60 anos       58849
<= 19 anos       33315
Name: Faixa Etária, dtype: int64


In [73]:
for i in b:
  print(i/len(df))

0.2481804391106279
0.22273330220391627
0.16764352635895005
0.1473373894499153
0.13671157201233097
0.07739377086425948


In [96]:
399/len(df)

0.0009269132395269258

In [54]:
def amostragem_estratificada(df, percentual):
  split = StratifiedShuffleSplit(test_size=percentual, random_state=1)
  for _, y in split.split(df, df['Faixa Etária']):
    df_y = df.iloc[y]
  return df_y

In [97]:
df_amostra_estratificada = amostragem_estratificada(df, 0.0009269132395269258)
df_amostra_estratificada.shape

(399, 18)

# Comparativo dos resultados

In [98]:
df['Idade'].mean()

41.060939318544534

In [99]:
df_amostra_aleatoria_simples['Idade'].mean()

41.82706766917293

In [100]:
df_amostra_sistematica['Idade'].mean()

41.9725

In [101]:
df_amostra_agrupamento['Idade'].mean()

40.9325

In [102]:
df_amostra_estratificada['Idade'].mean()

41.10526315789474